In [1]:
!pip install pycaret

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pycaret.datasets import get_data
df = get_data('Credit')

,Unnamed: 0,Default,checkingstatus1,duration,history,purpose,amount,savings,employ,installment,sex,residence,age,housing,cards,liable,tele,foreign
0,1,0,<DM0,6,terrible,goods/repair,1169,DM0-100,7+years,4,male,4,67,own,2,1,yes,foreign
1,2,1,DM0-200,48,poor,goods/repair,5951,DM0-100,1-7years,2,female,2,22,own,1,1,no,foreign
2,3,0,NoAccount,12,terrible,edu,2096,DM0-100,1-7years,2,male,3,49,own,1,2,no,foreign
3,4,0,<DM0,42,poor,goods/repair,7882,DM0-100,1-7years,2,male,4,45,forfree,1,2,no,foreign
4,5,1,<DM0,24,poor,newcar,4870,DM0-100,1-7years,3,male,4,53,forfree,2,2,no,foreign


In [3]:
df.shape

(1000, 18)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       1000 non-null   int64 
 1   Default          1000 non-null   int64 
 2   checkingstatus1  1000 non-null   object
 3   duration         1000 non-null   int64 
 4   history          1000 non-null   object
 5   purpose          1000 non-null   object
 6   amount           1000 non-null   int64 
 7   savings          1000 non-null   object
 8   employ           1000 non-null   object
 9   installment      1000 non-null   int64 
 10  sex              1000 non-null   object
 11  residence        1000 non-null   int64 
 12  age              1000 non-null   int64 
 13  housing          1000 non-null   object
 14  cards            1000 non-null   int64 
 15  liable           1000 non-null   int64 
 16  tele             1000 non-null   object
 17  foreign          1000 non-null   o

In [5]:
#checking for null values

df.isnull().sum()

Unnamed: 0         0
Default            0
checkingstatus1    0
duration           0
history            0
purpose            0
amount             0
savings            0
employ             0
installment        0
sex                0
residence          0
age                0
housing            0
cards              0
liable             0
tele               0
foreign            0
dtype: int64

### preprocessing

In [6]:
df.describe()

,Unnamed: 0,Default,duration,amount,installment,residence,age,cards,liable
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,0.300000,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000
std,288.819436,0.458487,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086
min,1.000000,0.000000,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,250.750000,0.000000,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,500.500000,0.000000,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000
75%,750.250000,1.000000,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000
max,1000.000000,1.000000,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000


In [7]:
!pip install pycaret

Defaulting to user installation because normal site-packages is not writeable


In [8]:
#label encoding

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
newdata = df.apply(le.fit_transform)
newdata.head

<bound method NDFrame.head of      Unnamed: 0  Default  checkingstatus1  duration  history  purpose  amount  \
0             0        0                0         2        2        2     142   
1             1        1                1        29        1        2     770   
2             2        0                3         8        2        1     390   
3             3        0                0        26        1        2     848   
4             4        1                0        17        1        3     734   
..          ...      ...              ...       ...      ...      ...     ...   
995         995        0                3         8        1        2     310   
996         996        0                0        21        1        4     661   
997         997        0                3         8        1        2      69   
998         998        1                0        27        1        2     332   
999         999        0                1        27        2        4     711  

In [9]:
Fraud = newdata[newdata['Default']==1]
Valid = newdata[newdata['Default']==0]
outlier_fraction = len(Fraud)/float(len(Valid))

In [10]:
print(outlier_fraction)
print("Fraud Cases : {}".format(len(Fraud)))
print("Valid Cases : {}".format(len(Valid)))

0.42857142857142855
Fraud Cases : 300
Valid Cases : 700


In [11]:
import numpy as np
columns = newdata.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Default"]]
# Store the variable we are predicting
target = "Default"
# Define a random state
state = np.random.RandomState(42)
X = newdata[columns]
Y = newdata[target]
X_outliers = state.uniform(low=0, high=1, size=(X.shape[0], X.shape[1]))
# Print the shapes of X & Y
print(X.shape)
print(Y.shape)

(1000, 17)
(1000,)


In [12]:
import warnings
warnings.filterwarnings("ignore")

### splitting dataset 

In [13]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2, random_state=42)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

(800, 17) (200, 17) (800,) (200,)


### Model Implementation 

In [14]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor


classifiers = {
    "Isolation Forest":IsolationForest(n_estimators=100, max_samples=len(X),
                                       contamination=0.5,random_state=42, verbose=0),
    "Local Outlier Factor":LocalOutlierFactor(n_neighbors=20, algorithm='auto',
                                              leaf_size=30, metric='minkowski',
                                              p=2, metric_params=None, contamination=0.5, novelty=True)

}

### Evaluation before sampling

In [15]:
from sklearn.metrics import classification_report,accuracy_score
n_outliers = len(Fraud)
for i, (clf_name,clf) in enumerate(classifiers.items()):
    #Fit the data and tag outliers
    if clf_name == "Local Outlier Factor":
        clf.fit(xtrain)
        scores_prediction = clf.negative_outlier_factor_
        y_pred = clf.predict(xtest)
    #elif clf_name == "Support Vector Machine":
    #    clf.fit(X)
    #    y_pred = clf.predict(X)
    else:
        clf.fit(xtrain)
        scores_prediction = clf.decision_function(xtrain)
        y_pred = clf.predict(xtest)
    #Reshape the prediction values to 0 for Valid transactions , 1 for Fraud transactions
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1
    n_errors = (y_pred != ytest).sum()
    # Run Classification Metrics
    print("{}: {}".format(clf_name,n_errors))
    print("Accuracy Score :")
    print(accuracy_score(ytest,y_pred))
    print("Classification Report :")
    print(classification_report(ytest,y_pred))

Isolation Forest: 103
Accuracy Score :
0.485
Classification Report :
              precision    recall  f1-score   support

           0       0.70      0.48      0.57       141
           1       0.29      0.51      0.37        59

    accuracy                           0.48       200
   macro avg       0.49      0.49      0.47       200
weighted avg       0.58      0.48      0.51       200

Local Outlier Factor: 101
Accuracy Score :
0.495
Classification Report :
              precision    recall  f1-score   support

           0       0.73      0.45      0.56       141
           1       0.32      0.61      0.42        59

    accuracy                           0.49       200
   macro avg       0.52      0.53      0.49       200
weighted avg       0.61      0.49      0.51       200



# Random undersampling

In [16]:
#RANDOM UNDERSAMPLING
from imblearn.under_sampling import RandomUnderSampler

# Perform random undersampling
rus = RandomUnderSampler(sampling_strategy=0.5, random_state=42)  # Adjust the sampling_strategy as needed
X_resampled, y_resampled = rus.fit_resample(X, Y)

print(X_resampled.shape, y_resampled.shape)

(900, 17) (900,)


### splitting data 

In [17]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

(720, 17) (180, 17) (720,) (180,)


### model implementation

In [18]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor


classifiers = {
    "Isolation Forest":IsolationForest(n_estimators=100, max_samples=len(X),
                                       contamination=0.5,random_state=42, verbose=0),
    "Local Outlier Factor":LocalOutlierFactor(n_neighbors=20, algorithm='auto',
                                              leaf_size=30, metric='minkowski',
                                              p=2, metric_params=None, contamination=0.5, novelty=True)

}

### Evaluation 

In [19]:
from sklearn.metrics import classification_report,accuracy_score
n_outliers = len(Fraud)
for i, (clf_name,clf) in enumerate(classifiers.items()):
    #Fit the data and tag outliers
    if clf_name == "Local Outlier Factor":
        clf.fit(xtrain)
        scores_prediction = clf.negative_outlier_factor_
        y_pred = clf.predict(xtest)
    #elif clf_name == "Support Vector Machine":
    #    clf.fit(X)
    #    y_pred = clf.predict(X)
    else:
        clf.fit(xtrain)
        scores_prediction = clf.decision_function(xtrain)
        y_pred = clf.predict(xtest)
    #Reshape the prediction values to 0 for Valid transactions , 1 for Fraud transactions
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1
    n_errors = (y_pred != ytest).sum()
    # Run Classification Metrics
    print("{}: {}".format(clf_name,n_errors))
    print("Accuracy Score :")
    print(accuracy_score(ytest,y_pred))
    print("Classification Report :")
    print(classification_report(ytest,y_pred))

Isolation Forest: 93
Accuracy Score :
0.48333333333333334
Classification Report :
              precision    recall  f1-score   support

           0       0.70      0.47      0.56       127
           1       0.29      0.51      0.37        53

    accuracy                           0.48       180
   macro avg       0.49      0.49      0.47       180
weighted avg       0.58      0.48      0.51       180

Local Outlier Factor: 90
Accuracy Score :
0.5
Classification Report :
              precision    recall  f1-score   support

           0       0.73      0.46      0.57       127
           1       0.31      0.58      0.41        53

    accuracy                           0.50       180
   macro avg       0.52      0.52      0.49       180
weighted avg       0.61      0.50      0.52       180



# Smote

In [20]:
from collections import Counter
from imblearn.over_sampling import SMOTE
print('Original dataset shape %s' % Counter(Y))
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, Y)
print('Resampled dataset shape %s' % Counter(y_res))

Original dataset shape Counter({0: 700, 1: 300})
Resampled dataset shape Counter({0: 700, 1: 700})


In [21]:
print(X_res.shape, y_res.shape)

(1400, 17) (1400,)


### splitting data 

In [22]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_res, y_res, test_size=0.2, random_state=42)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

(1120, 17) (280, 17) (1120,) (280,)


### model training 

In [23]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor


classifiers = {
    "Isolation Forest":IsolationForest(n_estimators=100, max_samples=len(X),
                                       contamination=0.5,random_state=42, verbose=0),
    "Local Outlier Factor":LocalOutlierFactor(n_neighbors=20, algorithm='auto',
                                              leaf_size=30, metric='minkowski',
                                              p=2, metric_params=None, contamination=0.5, novelty=True)

}

### Evaluation 

In [24]:
from sklearn.metrics import classification_report,accuracy_score
n_outliers = len(Fraud)
for i, (clf_name,clf) in enumerate(classifiers.items()):
    #Fit the data and tag outliers
    if clf_name == "Local Outlier Factor":
        clf.fit(xtrain)
        scores_prediction = clf.negative_outlier_factor_
        y_pred = clf.predict(xtest)
    #elif clf_name == "Support Vector Machine":
    #    clf.fit(X)
    #    y_pred = clf.predict(X)
    else:
        clf.fit(xtrain)
        scores_prediction = clf.decision_function(xtrain)
        y_pred = clf.predict(xtest)
    #Reshape the prediction values to 0 for Valid transactions , 1 for Fraud transactions
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1
    n_errors = (y_pred != ytest).sum()
    # Run Classification Metrics
    print("{}: {}".format(clf_name,n_errors))
    print("Accuracy Score :")
    print(accuracy_score(ytest,y_pred))
    print("Classification Report :")
    print(classification_report(ytest,y_pred))

Isolation Forest: 179
Accuracy Score :
0.3607142857142857
Classification Report :
              precision    recall  f1-score   support

           0       0.39      0.35      0.37       149
           1       0.34      0.37      0.35       131

    accuracy                           0.36       280
   macro avg       0.36      0.36      0.36       280
weighted avg       0.36      0.36      0.36       280

Local Outlier Factor: 167
Accuracy Score :
0.4035714285714286
Classification Report :
              precision    recall  f1-score   support

           0       0.43      0.35      0.38       149
           1       0.39      0.47      0.42       131

    accuracy                           0.40       280
   macro avg       0.41      0.41      0.40       280
weighted avg       0.41      0.40      0.40       280



# Smote-ENN

In [25]:
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours
from collections import Counter

sampler = SMOTEENN(random_state=42)
X_resampled, y_resampled = sampler.fit_resample(X, Y)

print(X_resampled.shape, y_resampled.shape)

(505, 17) (505,)


### splitting dataset 

In [26]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

(404, 17) (101, 17) (404,) (101,)


### model training

In [27]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor


classifiers = {
    "Isolation Forest":IsolationForest(n_estimators=100, max_samples=len(X),
                                       contamination=0.5,random_state=42, verbose=0),
    "Local Outlier Factor":LocalOutlierFactor(n_neighbors=20, algorithm='auto',
                                              leaf_size=30, metric='minkowski',
                                              p=2, metric_params=None, contamination=0.5, novelty=True)

}

### evaluation

In [28]:
from sklearn.metrics import classification_report,accuracy_score
n_outliers = len(Fraud)
for i, (clf_name,clf) in enumerate(classifiers.items()):
    #Fit the data and tag outliers
    if clf_name == "Local Outlier Factor":
        clf.fit(xtrain)
        scores_prediction = clf.negative_outlier_factor_
        y_pred = clf.predict(xtest)
    #elif clf_name == "Support Vector Machine":
    #    clf.fit(X)
    #    y_pred = clf.predict(X)
    else:
        clf.fit(xtrain)
        scores_prediction = clf.decision_function(xtrain)
        y_pred = clf.predict(xtest)
    #Reshape the prediction values to 0 for Valid transactions , 1 for Fraud transactions
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1
    n_errors = (y_pred != ytest).sum()
    # Run Classification Metrics
    print("{}: {}".format(clf_name,n_errors))
    print("Accuracy Score :")
    print(accuracy_score(ytest,y_pred))
    print("Classification Report :")
    print(classification_report(ytest,y_pred))

Isolation Forest: 74
Accuracy Score :
0.26732673267326734
Classification Report :
              precision    recall  f1-score   support

           0       0.20      0.28      0.23        40
           1       0.36      0.26      0.30        61

    accuracy                           0.27       101
   macro avg       0.28      0.27      0.27       101
weighted avg       0.29      0.27      0.27       101

Local Outlier Factor: 54
Accuracy Score :
0.46534653465346537
Classification Report :
              precision    recall  f1-score   support

           0       0.37      0.47      0.41        40
           1       0.57      0.46      0.51        61

    accuracy                           0.47       101
   macro avg       0.47      0.47      0.46       101
weighted avg       0.49      0.47      0.47       101

